<a href="https://colab.research.google.com/github/aman9394/aman9394/blob/main/Tesseract_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

In [3]:
!pip install dateparser

     |████████████████████████████████| 286kB 5.2MB/s 


In [5]:
!pip install pdf2image

In [12]:
import cv2
import pytesseract
from pytesseract import Output
import re
from pprint import pprint
from dateparser.search import search_dates
from pdf2image import convert_from_path
import time

In [15]:
def detect_page_layout(path):
    # Image read
    img_ = cv2.imread(path)
    gray = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # Config for tesseract
    custom_config = r'-l eng --oem 1 --psm 1 textord_tabfind_show_blocks 1 '
    # Tesseract to extract data from image
    d = pytesseract.image_to_data(thresh, config=custom_config, output_type=Output.DICT)

    # Init variables
    prev_block = 0
    prev_line = 0
    prev_line_index = 0
    prev_word_index = 0
    
    top_left_point = {
        "x": 0,
        "y": 0
    }
    top_right_point = {
        "x": 0,
        "y": 0
    }
    bottom_left_point = {
        "x": 0,
        "y": 0
    }
    highest_right_x = 0
    first_line_drawn = False
    outputDict = []
    block = ""
    

    # Iterate over every word identified
    n_boxes = len(d['text'])
    for i in range(n_boxes):
        print(d["text"][i])
        
        # If previous block is not set. To be executed only first time
        if prev_block == 0 :
            prev_block = d['block_num'][i]
            top_left_point = {
                "x": d['left'][i],
                "y": d['top'][i],
                "w": d['width'][i],
                "h": d['height'][i],
            }
        
        # check if block has changed
        if prev_block != d["block_num"][i]:
            prev_block = d["block_num"][i]
            
            if not first_line_drawn :
                if prev_word_index :
                    top_right_point = {
                        "x": d['left'][prev_word_index],
                        "y": d['top'][prev_word_index],
                        "w": d['width'][prev_word_index],
                        "h": d['height'][prev_word_index],
                    }
                    first_line_drawn = True
                    # Drawing top line from left to right
                    img_ = cv2.line(img_, (top_left_point["x"], top_left_point["y"]), (top_right_point["x"] + top_right_point["w"] , top_right_point["y"]), highlightColor, 2)
                    # Drawing left line (in case of single line block) from left to bottom
                    img_ = cv2.line(img_, (top_left_point["x"], top_left_point["y"]), (top_left_point["x"], top_left_point["y"] + top_left_point['h'] ), highlightColor, 2)
                    bottom_left_point = {
                        "x": d['left'][first_word_index],
                        "y": d['top'][first_word_index],
                        "w": d['width'][first_word_index],
                        "h": d['height'][first_word_index],
                    }
                    bottom_right_point = {
                        "x": d['left'][prev_word_index],
                        "y": d['top'][prev_word_index],
                        "w": d['width'][prev_word_index],
                        "h": d['height'][prev_word_index],
                    }
                    
                    # Drawing bottom line from left to right
                    img_ = cv2.line(img_, (bottom_left_point["x"], bottom_left_point["y"] + bottom_left_point["h"] ), (bottom_right_point["x"] + bottom_right_point["w"], bottom_right_point["y"] + bottom_right_point["h"] ), highlightColor, 2)
                    
                    outputDict.append({
                        "block_text": block,
                        "coordinates": [
                            {
                                "x": top_left_point["x"],
                                "y": top_left_point["y"]
                            },
                            {
                                "x": bottom_right_point["x"] + bottom_right_point["w"],
                                "y": bottom_right_point["y"] + bottom_right_point["h"]
                            }
                        ]
                    })
            else:
                bottom_left_point = {
                    "x": d['left'][first_word_index],
                    "y": d['top'][first_word_index],
                    "w": d['width'][first_word_index],
                    "h": d['height'][first_word_index],
                }

                # Drawing left line from left to bottom of block
                img_ = cv2.line(img_, (top_left_point["x"], top_left_point["y"]), (top_left_point["x"], bottom_left_point["y"] + bottom_left_point["h"] ), highlightColor, 2)

                if highest_right_x > d['left'][prev_word_index] & False:
                    bottom_right_point = {
                        "x": highest_right_x,
                        "y": d['top'][prev_word_index],
                        "w": d['width'][prev_word_index],
                        "h": d['height'][prev_word_index],
                    }
                else :
                    bottom_right_point = {
                        "x": d['left'][prev_word_index],
                        "y": d['top'][prev_word_index],
                        "w": d['width'][prev_word_index],
                        "h": d['height'][prev_word_index],
                    }
                
                
                if (highest_right_x > top_right_point["x"]) :
                    # Drawing top line from left to right
                    img_ = cv2.line(img_, (top_left_point["x"], top_left_point["y"]), (highest_right_x, top_right_point["y"]), highlightColor, 2)
                    # Drawing bottom line from left to right
                    img_ = cv2.line(img_, (top_left_point["x"], bottom_left_point["y"] + bottom_left_point["h"] ), (highest_right_x, bottom_right_point["y"] + bottom_right_point["h"] ), highlightColor, 2)
                    # Drawing right line from right top to bottom right
                    img_ = cv2.line(img_, (highest_right_x, top_right_point["y"] ), (highest_right_x, bottom_right_point["y"] + bottom_right_point["h"] ), highlightColor, 2)
                    outputDict.append({
                        "block_text": block,
                        "coordinates": [
                            {
                                "x": top_left_point["x"],
                                "y": top_left_point["y"]
                            },
                            {
                                "x": highest_right_x,
                                "y": bottom_right_point["y"] + bottom_right_point["h"]
                            }
                        ]
                    })   
                else:
                    # Drawing bottom line from left to right
                    img_ = cv2.line(img_, (top_left_point["x"], bottom_left_point["y"] + bottom_left_point["h"] ), (top_right_point["x"] + top_right_point["w"], bottom_right_point["y"] + bottom_right_point["h"] ), highlightColor, 2)
                    # Drawing right line from right top to bottom right
                    img_ = cv2.line(img_, (top_right_point["x"] + top_right_point["w"], top_right_point["y"] ), (top_right_point["x"] + top_right_point["w"], bottom_right_point["y"] + bottom_right_point["h"] ), highlightColor, 2)
                    outputDict.append({
                        "block_text": block,
                        "coordinates": [
                            {
                                "x": top_left_point["x"],
                                "y": top_left_point["y"]
                            },
                            {
                                "x": top_right_point["x"] + top_right_point["w"],
                                "y": bottom_right_point["y"] + bottom_right_point["h"]
                            }
                        ]
                    })  

            block = ""          
            first_line_drawn = False
            first_word_index = i
            prev_word_index = False
            highest_right_x = 0
            top_left_point = {
                "x": d['left'][i],
                "y": d['top'][i],
                "w": d['width'][i],
                "h": d['height'][i],
            }

        # check when line changes
        if prev_line != d["line_num"][i] :
            block += "\n"
            prev_line = d["line_num"][i]
            first_word_index = i

            if (prev_word_index):
                if not first_line_drawn :
                    top_right_point = {
                        "x": d['left'][prev_word_index],
                        "y": d['top'][prev_word_index],
                        "w": d['width'][prev_word_index],
                        "h": d['height'][prev_word_index]
                    }
                        
                    first_line_drawn = True
                    # Drawing top line from left to right
                    highlightColor = ( 0, 0, 255 )
                    img_ = cv2.line(img_, (top_left_point["x"], top_left_point["y"]), (top_right_point["x"] + + d['width'][prev_word_index], top_right_point["y"]), highlightColor, 2)
        # the highest right point in a given block
        if highest_right_x < d['left'][i] :
            highest_right_x = d['left'][i] +  d['width'][i] 
        prev_word_index = i
        block += d["text"][i] + " "

    cv2.imwrite('img.jpeg', img_)
    return outputDict
    


In [ ]:
text = detect_page_layout('./sample_data/input.jpg')

In [17]:
img_ = cv2.imread('./sample_data/input.jpg')
gray = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [39]:
d = pytesseract.image_to_string(img_)

In [40]:
d

'Squarepants, Spongebob 11/4/2020 Office Visit - Henry Ford Health System\n\nSpongebob Squarepants\n\nEncounter Summary, generated on Dec. 30, 2020\n\nPrintout Information\nDocument Contents Document Received Date Document Source Organization\n\n \n\nOffice Visit Dec. 30, 2020 Henry Ford Health System\n\nPatient Demographics -7y.0. Male; born Dec. 30, 2013\n\nPatient Address © Communication Language Race/ Ethnicity Marital Status\nEnglish Unknown/ Single\n(Preferred) Unknown\n\nReason for Visit\nReason Comments\nEstablish Care\n\n \n\nEncounter Details\n\nDate Type Department Care Team Description\n11/04/2020 Office Visit Sunshine Kremer MD White, Sunshine FNP Encounter for routine\nPLLC 2195 SPRING 2195 SPRING ARBOR RD child health examination\nARBOR RDJACKSON, M_ JACKSON, MI 49203- without abnormal\n49203-2748 2748 findings (Primary Dx);\n517-539-6111 517-539-6111 Dietary counseling;\n517-539-6110 (Fax) Exercise counseling;\n\nBMI (body mass index),\npediatric, 5% to less than\n85% f

In [36]:
def detect_social_history(data, path):
    # print(data)
    highlightColor = ( 0, 0, 255 )
    for block in data:
        block_text = block["block_text"].lower()
        Result = re.findall("social history.*",block_text)
        
        if (len(Result)):
            img_ = cv2.imread(path)
            img_ = cv2.rectangle(img_, (block["coordinates"][0]["x"], block["coordinates"][0]["y"]), (block["coordinates"][1]["x"], block["coordinates"][1]["y"]), highlightColor, 10)
            cv2.imwrite("output/output" + str(time.time()) + ".jpg", img_)

In [37]:
detect_social_history(text, './sample_data/input.jpg')

In [38]:
text

[{'block_text': '   \n Squarepants, Spongebob 11/4/2020 Office Visit - Henry Ford Health System ',
  'coordinates': [{'x': 516, 'y': 43}, {'x': 5025, 'y': 153}]},
 {'block_text': '\n  \n Spongebob Squarepants ',
  'coordinates': [{'x': 610, 'y': 666}, {'x': 1867, 'y': 799}]},
 {'block_text': '\n  \n Encounter Summary, generated on Dec. 30, 2020 ',
  'coordinates': [{'x': 606, 'y': 829}, {'x': 2133, 'y': 888}]},
 {'block_text': '\n  \n Printout Information \n Document Contents Document Received Date Document Source Organization ',
  'coordinates': [{'x': 708, 'y': 1046}, {'x': 4083, 'y': 1318}]},
 {'block_text': '\n  \n   ',
  'coordinates': [{'x': 518, 'y': 1319}, {'x': 4970, 'y': 1334}]},
 {'block_text': '\n  \n Office Visit Dec. 30, 2020 Henry Ford Health System ',
  'coordinates': [{'x': 703, 'y': 1361}, {'x': 3871, 'y': 1454}]},
 {'block_text': '\n  \n Patient Demographics -7y.0. Male; born Dec. 30, 2013 ',
  'coordinates': [{'x': 708, 'y': 1595}, {'x': 3302, 'y': 1699}]},
 {'block